# BEE 4750 Final Project: Solar Photovoltaic Feasibility Study on Cornell's Engineering Quadrangle

**Names**: Sitara Sastry, Maya Zoe Yu

**ID**: sls496, mzy3

> **Introduction**
>
> The following model will conduct a feasibility study of solar energy production on Cornell's Engineering Quadrangle. As Cornell’s Climate Action Plan seeks to have Cornell’s Ithaca campus carbon neutral by 2035 through utilizing 100% renewable energy, the proposed research will analyze if solar energy would help to achieve these goals. As such, the study will analyze how Cornell can invest in solar photovoltaics (PV) to increase this percentage – focusing mainly on the Engineering Quadrangle. In addition, the model will analyze the theoretical production of solar rooftop PV systems while optimzing cost. 


### Theoretical Solar Output Model

#### Electricity Demand for Cornell's Engineering Quad
Cornell's Engineering Quadrangle encompasses the following buildings: Olin Hall, Carpenter Hall, Duffield Hall, Phillips Hall, Rhodes Hall, Upson Hall, Kimball Hall, Thurston Hall, Bard Hall, Snee Hall, and Hollister Hall. The yearly electricity demand for these buildings may be found through the following: 

In [3]:
### Solar Geometry Function
function solar_geometry(N, L, H, β, γ)
    δ = 23.45*sin[(360(284+N))/365] # Declination Angle 
    ω = (H-12)*15 # Hour Angle 
    α = asin[sin(δ)*sin(L)+cos(δ)*cos(L)*cos(ω)] # Solar Altitude Angle
    γ_s = asin[(cos(δ)*sin(ω))/cos(α)] # Solar Azimuth Angle
    θ_i = acos(sin(α)*cos(β)+cos(α)*sin(β)*cos(γ-γ_s)) # Incidence Angle
end

### Solar Insolation - Direct, Diffuse, Reflected
function solar_insolation(N, L, β, K_t, ρ, H)
    ratio_diffuse_global = 1.39-4.03(K_t)+5.53(K_t)^2-3.11(K_t)^3 # Ratio of Diffuse to Global Insolation
    # Calculate sunset hour angle based on N
    if N >= 79 && N <= 265
        ω_s = min(acos(-tan(L)*tan(δ)), acos(-tan(L-β)*tan(δ)))
    else 
        ω_s = acos(-tan(L)*tan(δ))
    end
    # Ratio of Beam at Angle to Beam onto Horizontal Surface
    R_{b,β}=[cos(L-β)cos(δ)sin(ω_s)+ω_ssin(L-β)sin(δ)]/[cos(L)cos(δ)sin(ω_s)+ω_ssin(L)sin(δ)] 
    # Calculate direct, diffuse, and reflected insolation
    Direct = (1-ratio_diffuse_global)R_{b,β}
    Diffuse = (ratio_diffuse_global)[1+cos(β)]/2
    Reflected = ρ(1-cos(β))/2
    # Calcuate ratio of global insolation at angle to horizontal
    R = Direct+Diffuse+Reflected
    # Calculate total insolation onto angled surface at angle β [Energy/Area]
    H_t = H(R)
end



solar_insolation (generic function with 3 methods)

#### Future Steps 
- Sitara: Decide area for each building 
- Collect 8760 for each building 
- Assume beta, L, K_t, rho, H, gamma 
- Create data frame for N (1:365)
